# 🧹 LIMPIEZA GETYOURGUIDE

In [ ]:
!pip install emoji dateparser langdetect

In [ ]:
import os, glob, re
import pandas as pd
from functools import lru_cache
from datetime import datetime

In [ ]:
# ==========================
# Limpieza + Unificación GetYourGuide (optimizado y con mayúsculas/minúsculas)
# ==========================

# ---------- Imports ----------
import os
import glob
import re
import pandas as pd
from functools import lru_cache
import dateparser

# ---------- Config ----------
DATA_DIR = os.getcwd()  # misma carpeta del notebook
FILE_PATTERN = os.path.join(DATA_DIR, "experiencias_reviews_getyourguide_*.csv")
files = glob.glob(FILE_PATTERN)

print("Archivos encontrados:")
for f in files:
    print(" -", os.path.basename(f))

# ---------- Emojis ----------
EMOJI_RE = re.compile(
    "["
    "\U0001F300-\U0001F5FF"
    "\U0001F600-\U0001F64F"
    "\U0001F680-\U0001F6FF"
    "\U0001F700-\U0001F77F"
    "\U0001F780-\U0001F7FF"
    "\U0001F900-\U0001F9FF"
    "\U00002600-\U000026FF"
    "\U00002700-\U000027BF"
    "\U00002B00-\U00002BFF"
    "]+"
)

# Al poner esto en True, se convierte cada emoji en su descripción textual
USE_DEMOJIZE = True
try:
    import emoji
    HAS_EMOJI = True
except Exception:
    HAS_EMOJI = False

def limpiar_emoji(texto: str) -> str:
    """
    Si USE_DEMOJIZE es True y está instalada la librería emoji,
    reemplaza cada emoji por su descripción en español (sin dos puntos ni guiones bajos).
    En caso contrario, elimina el emoji por completo.
    """
    if not isinstance(texto, str):
        return ""
    if USE_DEMOJIZE and HAS_EMOJI and EMOJI_RE.search(texto):
        # Quita emoticonos y limpia: :cara_sonriente: -> "cara sonriente"
        t = emoji.demojize(texto, language="es").replace(":", "").replace("_", " ")
        return re.sub(r"\s+", " ", t).strip()
    return re.sub(r"\s+", " ", EMOJI_RE.sub("", texto)).strip()

# ---------- Normalización de mayúsculas/minúsculas ----------
def normalizar_mayusculas(texto: str) -> str:
    """Convierte el texto a minúsculas y capitaliza el inicio y tras ., ?, !, ;"""
    if not isinstance(texto, str):
        return ""
    texto = texto.strip().lower()
    def _capitalize(match):
        return match.group(0).upper()
    texto = re.sub(r'(^[a-záéíóúüñ])', _capitalize, texto)
    texto = re.sub(
        r'([\.!?;]\s*)([a-záéíóúüñ])',
        lambda m: m.group(1) + m.group(2).upper(),
        texto
    )
    return texto

# ---------- Sentimiento ----------
def sentimiento_from_score(x):
    """Clasifica el sentimiento a partir de un valor numérico (1–10)"""
    try:
        v = float(x)
    except Exception:
        return "Desconocido"
    if 1 <= v <= 2: return "Malo"
    if v == 3:      return "Neutro"
    if 4 <= v <= 5: return "Bueno"
    if 1 <= v <= 4:  return "Malo"
    if 5 <= v <= 6:  return "Neutro"
    if 7 <= v <= 10: return "Bueno"
    return "Desconocido"

# ---------- Experiencia ----------
EXPERIENCE_KEYWORDS = {
    "Hotel": ["hotel","hostal","albergue","inn","auberge","hôtel","pousada","pensione","otel","hostel"],
    "Playa": ["playa","beach","spiaggia","praia","strand","plage","chiringuito"],
    "Tour": ["tour","excursion","excursión","ruta","visita guiada","guided tour","excursao","gira","itinerario","paseo","guia"],
    "Deporte": ["deporte","deportes","sport","sports","surf","tenis","fútbol","football","soccer","gym","gimnasio","baloncesto","pádel","paddle","ciclismo","hiking","senderismo", "kayak","snorkel"],
    "Cultura": ["cultura","cultural","culture","teatro","ópera","opera","arquitectura","historia","patrimonio","heritage","monumento","exposición","exposicion"],
    "Comida": ["comida","food","gastronomía","gastronomia","restaurante","restaurant","cena","almuerzo","desayuno","tapas","platos","dishes","cuisine","eat","manger","mangiare"],
    "Museo": ["museo","museum","musée","museu","musei","galería","galeria","gallery"],
    "Vida nocturna": ["discoteca","nightlife","clubbing","fiesta","pub","bares","copas","afterwork","club","night club"],
    "Ocio": ["ocio","entretenimiento","diversión","leisure","entertainment","evento","eventos","planes","show", "taller","activity"],
}
EXPERIENCE_ORDER = ["Hotel","Playa","Tour","Deporte","Cultura","Comida","Museo","Vida nocturna","Ocio"]

# Precompilamos patrones para búsquedas más rápidas
EXP_PATTERNS = {cat: r"(?:" + r"|".join(map(re.escape, kws)) + r")"
                for cat, kws in EXPERIENCE_KEYWORDS.items()}

def clasificar_experiencia_vectorizado(serie_texto: pd.Series) -> pd.Series:
    """Clasifica la experiencia según palabras clave en orden de prioridad"""
    flags = {}
    for cat in EXPERIENCE_ORDER:
        flags[cat] = serie_texto.str.contains(EXP_PATTERNS[cat], case=False, na=False, regex=True)
    flags_df = pd.DataFrame(flags, index=serie_texto.index)
    result = pd.Series("Desconocido", index=serie_texto.index)
    mask_restante = pd.Series(True, index=serie_texto.index)
    for cat in EXPERIENCE_ORDER:
        m = mask_restante & flags_df[cat]
        result[m] = cat
        mask_restante &= ~flags_df[cat]
        if not mask_restante.any():
            break
    return result

# ---------- Helper para parsear fechas en formato DD/MM/AAAA ----------
def parse_fecha_ddmmaaaa(valor: str):
    """
    Convierte una cadena de fecha a formato DD/MM/AAAA usando dateparser.
    Si no se puede parsear, devuelve pd.NA.
    """
    if pd.isna(valor):
        return pd.NA
    s = str(valor).strip()
    if not s:
        return pd.NA
    # Intentamos parsear con dateparser (soporta varios idiomas)
    dt = dateparser.parse(s, settings={
        "PREFER_DAY_OF_MONTH": "first",
        "PREFER_DATES_FROM": "past",
        "RETURN_AS_TIMEZONE_AWARE": False,
        "SKIP_TOKENS": ["de","del","en","da","di","em","im","nel","na","no","of"]
    })
    if dt:
        return dt.strftime("%d/%m/%Y")
    # Como última opción, probamos con pandas
    try:
        dt = pd.to_datetime(s, dayfirst=True)
        return dt.strftime("%d/%m/%Y")
    except Exception:
        return pd.NA

# ---------- Localizador de columnas ----------
def get_col(df, candidates):
    """Devuelve el nombre real de la columna que coincide con cualquiera de los candidatos"""
    cols = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand.lower() in cols:
            return cols[cand.lower()]
    for c in df.columns:
        for cand in candidates:
            if cand.lower() in c.lower():
                return c
    return None

# ---------- Normalizar ciudad desde el nombre de archivo ----------
def _city_from_filename(fname: str) -> str:
    m = re.match(r"experiencias_reviews_getyourguide_(.+?)\.csv$", os.path.basename(fname), re.IGNORECASE)
    city = (m.group(1) if m else "").strip().lower()
    mapping = {
        "grancanaria": "Gran Canaria",
        "canaria": "Gran Canaria",
        "malaga": "Málaga",
        "barcelona": "Barcelona",
        "madrid": "Madrid",
        "mallorca": "Mallorca",
        "sevilla": "Sevilla",
        "tenerife": "Tenerife",
        "valencia": "Valencia",
    }
    return mapping.get(city, city.capitalize())

# ---------- Proceso por archivo ----------
def process_file(filepath: str) -> pd.DataFrame:
    """Limpia y normaliza un archivo CSV de GetYourGuide, devolviendo un DataFrame unificado"""
    city = _city_from_filename(filepath)
    df = pd.read_csv(filepath, encoding="utf-8", on_bad_lines="skip")

    col_coment = get_col(df, ["comentario","comentarios","review_text","comment","texto","text"])
    col_score  = get_col(df, ["review_score"])
    col_titulo = get_col(df, ["titulo","título","title","review_title"])
    col_fecha  = get_col(df, ["fecha","date","review_date","posted"])

    if not col_coment:
        raise ValueError(f"No se encontró la columna de comentarios en {os.path.basename(filepath)}")

    # Limpieza de texto
    texto_crudo  = df[col_coment].astype(str)
    # Demojizamos y normalizamos
    texto_limpio = texto_crudo.map(limpiar_emoji).map(normalizar_mayusculas)

    # Sentimiento
    if col_score:
        sentimiento = df[col_score].map(sentimiento_from_score)
    else:
        sentimiento = pd.Series(["Desconocido"] * len(df), index=df.index)

    # Clasificación de experiencia
    if col_titulo:
        combinado = (df[col_titulo].astype(str).fillna("") + " " + texto_crudo).str.strip()
    else:
        combinado = texto_crudo
    experiencia = clasificar_experiencia_vectorizado(combinado)

    # Fecha: parseamos y formateamos a DD/MM/AAAA si existe la columna, si no NA
    if col_fecha:
        fecha = df[col_fecha].apply(parse_fecha_ddmmaaaa)
    else:
        fecha = pd.Series([pd.NA] * len(df), index=df.index)

    # Limpieza del título: demojizamos, eliminamos la ciudad al inicio y, si quieres, normalizamos
    if col_titulo:
        titulo_series = df[col_titulo].astype(str)
        # Primero convertimos emojis a texto
        titulo_series = titulo_series.map(limpiar_emoji)
        # Si quieres normalizar mayúsculas en los títulos, descomenta la siguiente línea:
        # titulo_series = titulo_series.map(normalizar_mayusculas)
        # Eliminamos el prefijo de ciudad (por ejemplo "Madrid: " o "madrid-")
        patron_ciudad = re.compile(
            rf'^\s*{re.escape(city)}\s*[:\-–]\s*',
            flags=re.IGNORECASE
        )
        titulo_limpio = titulo_series.str.replace(patron_ciudad, '', regex=True)
    else:
        titulo_limpio = ""

    # DataFrame de salida
    out = pd.DataFrame({
        "Titulo": titulo_limpio,
        "Texto": texto_limpio,
        "Sentimiento": sentimiento,
        "Ciudad": city,
        "Experiencia": experiencia,
        "Fecha": fecha,
        "Fuente": "Get Your Guide"
    })
    return out

# ---------- Ejecutar ----------
processed = []
errores = {}

for fp in files:
    try:
        df_out = process_file(fp)
        processed.append(df_out)
        print("OK:", os.path.basename(fp), "->", len(df_out), "filas")
    except Exception as e:
        errores[os.path.basename(fp)] = str(e)
        print("ERROR:", os.path.basename(fp), "->", e)

if processed:
    final_df = pd.concat(processed, ignore_index=True)
    out_path = os.path.join(DATA_DIR, "getyourguide_unificado.csv")
    final_df.to_csv(out_path, index=False, encoding="utf-8")
    print("\nArchivo final guardado en:", out_path, "| Filas:", len(final_df))
else:
    print("No se generó salida.")

if errores:
    print("\nArchivos con error:")
    for k, v in errores.items():
        print(" -", k, ":", v)

# Vista rápida (solo tiene efecto en un notebook)
try:
    display(final_df.head())
except Exception:
    pass


Archivos encontrados:
 - experiencias_reviews_getyourguide_malaga.csv
 - experiencias_reviews_getyourguide_barcelona.csv
 - experiencias_reviews_getyourguide_madrid.csv
 - experiencias_reviews_getyourguide_sevilla.csv
 - experiencias_reviews_getyourguide_mallorca.csv
 - experiencias_reviews_getyourguide_canaria.csv
 - experiencias_reviews_getyourguide_tenerife.csv
 - experiencias_reviews_getyourguide_Valencia.csv
OK: experiencias_reviews_getyourguide_malaga.csv -> 2047 filas
OK: experiencias_reviews_getyourguide_barcelona.csv -> 4601 filas
OK: experiencias_reviews_getyourguide_madrid.csv -> 6114 filas
OK: experiencias_reviews_getyourguide_sevilla.csv -> 3150 filas
OK: experiencias_reviews_getyourguide_mallorca.csv -> 912 filas
OK: experiencias_reviews_getyourguide_canaria.csv -> 1480 filas
OK: experiencias_reviews_getyourguide_tenerife.csv -> 4521 filas
OK: experiencias_reviews_getyourguide_Valencia.csv -> 1088 filas

Archivo final guardado en: /Users/thaliacamargo/getyourguide_unifica

,Titulo,Texto,Sentimiento,Ciudad,Experiencia,Fecha,Fuente
0,Paseo en Catamarán a Vela con Opción Puesta de...,"El paseo a merecido la pena,los guías majisimo...",Bueno,Málaga,Tour,06/08/2025,Get Your Guide
1,Paseo en Catamarán a Vela con Opción Puesta de...,Compré dos tickets para ir sentados y muy bien...,Bueno,Málaga,Tour,20/07/2025,Get Your Guide
2,Paseo en Catamarán a Vela con Opción Puesta de...,"El viaje estuvo muy bien, tuvimos oportunidad ...",Bueno,Málaga,Tour,04/05/2025,Get Your Guide
3,Paseo en Catamarán a Vela con Opción Puesta de...,"Nos gustó mucho, es un paseo hermoso . Súper r...",Bueno,Málaga,Tour,19/04/2025,Get Your Guide
4,Paseo en Catamarán a Vela con Opción Puesta de...,Servicio en el bote muy atento. Buenos precios...,Bueno,Málaga,Tour,17/04/2025,Get Your Guide


In [ ]:
# Conteo por ciudad y experiencia
conteo = final_df.groupby(["Ciudad","Experiencia"]).size().reset_index(name="Cantidad")

# Si prefieres ver como tabla pivote (más legible)
pivot = conteo.pivot(index="Ciudad", columns="Experiencia", values="Cantidad").fillna(0).astype(int)
print(pivot)


Experiencia   Comida  Cultura  Deporte  Desconocido  Hotel  Museo  Ocio  \
Ciudad                                                                    
Barcelona        193      138       88         1277     80    114    73   
Gran Canaria      38        6       63          308     48      0     6   
Madrid           330      295       57          599     32    343   192   
Mallorca          44        8       42          143     22      5     5   
Málaga            49       29        5          279     17     35     6   
Sevilla           72      119       30          551     41     24    46   
Tenerife         128       20      482         1141     75      2    40   
Valencia          84       18       19          239     18     25    24   

Experiencia   Playa  Tour  Vida nocturna  
Ciudad                                    
Barcelona        64  2554             20  
Gran Canaria    103   904              4  
Madrid           12  4247              7  
Mallorca         32   597         